# Название тренировочного и тестового датасетов

In [51]:
training_dataset_name = 'train2023.csv'
testing_dataset_name = 'test2023.csv'

# Чтение датасетов из csv-файлов с помощью библиотеки pandas

In [52]:
import os
import pandas as pd

train_df = pd.read_csv(os.path.join('data', training_dataset_name), sep=';', header=None)
test_df = pd.read_csv(os.path.join('data', testing_dataset_name), sep=';', header=None)

classes = [1, 2, 3]

# Настройка логгирования

In [53]:
import logging

logging.basicConfig(level=logging.INFO)

# `OutlierTransformer`
Обертка над классом детектора выбросов. Необходим по причине того, что библиотека scikit-learn предполагает использование детектора выбросов не для преобработки датасета, а только в качестве классификатора. Метод `fit_transform()` очищает тренировочную выборку от выбросов, см. комментарии в коде.

In [54]:
import numpy as np


class OutlierTransformer:

    def __init__(self, outlier_detector, class_labels):
        self._class_labels = class_labels
        self._outlier_detector = outlier_detector

    def fit_transform(self, X, y, logging_level):
        # Размерность массива X до очистки от выбросов
        before = X.shape
        # Разделение объектов массива X по классам
        X_separated_by_class = {i: X[y == i, :] for i in self._class_labels}
        # Непосредственно очистка массива X от выбросов c помощью объекта библиотечного класса self._outlier_detector
        X_separated_by_class_cleared = {i:
                                            X_separated_by_class[i][
                                            self._outlier_detector.fit_predict(X=X_separated_by_class[i]) == 1, :] for i
                                        in
                                        self._class_labels}
        # Формирование массива X, очищенного от выбросов
        X = np.vstack(list(X_separated_by_class_cleared.values()))
        # Размерность массива X после очистки от выбросов
        after = X.shape
        # Формирование нового массива целевых признаков
        y = np.hstack([np.full(X_separated_by_class_cleared[i].shape[0], i) for i in self._class_labels])
        logging.log(level=logging_level, msg=f'CLEARING OUTLIERS: {before} -> {after}')

        return X, y

# `Scheme`
Обертка над библиотечным классом `Pipiline`, нужна главным образом для того, чтобы использовать определенный выше класс `OutlierTransformer`, так как библиотека не предполагает, что "трансформеры" будет изменять размер датасета.

In [55]:
from sklearn.pipeline import Pipeline


class Scheme:

    def __init__(self, pipeline: Pipeline, outlier_detector=None, class_labels=classes, hyperparams_str: str = None):
        if outlier_detector is not None:
            self._outlier_transformer = OutlierTransformer(outlier_detector=outlier_detector, class_labels=classes)
        else:
            self._outlier_transformer = None
        self._pipeline = pipeline

        self.hyperparams = hyperparams_str

    def fit(self, X, y, logging_level=logging.INFO):
        if self._outlier_transformer is not None:
            X, y = self._outlier_transformer.fit_transform(X=X, y=y, logging_level=logging_level)
        self._pipeline.fit(X=X, y=y)

    def predict(self, X):
        return self._pipeline.predict(X=X)
    
    def predict_proba(self, X): 
        return np.max(self._pipeline.predict_proba(X=X), axis=1)

# Обучение гиперпараметров
Функция `tune_hyperparams()` ответственна за обучение гиперпараметров. Обучение происходит с помощью кросс-валидации с количество разбиений по умолчанию равным 5. Лучшая схема - та, которой соответствует наибольшее среднее значение по всем разбиениям метрики `metric`.   

In [56]:
from sklearn.model_selection import StratifiedKFold


def tune_hyperparams(schemas, X, y, metric: callable, n_splits: int = 5):
    skf = StratifiedKFold(n_splits=n_splits)
    best_scheme = None
    best_score = None
    for scheme in schemas:
        score = 0
        logging.debug(scheme.hyperparams)
        for train, valid in skf.split(X, y):
            scheme.fit(X=X[train], y=y[train], logging_level=logging.DEBUG)
            y_predict = scheme.predict(X[valid])
            score += metric(y[valid], y_predict)
        if best_score is None or score > best_score:
            best_score = score
            best_scheme = scheme
        logging.debug('metric = ' + str(score / n_splits))
    return best_scheme, best_score / n_splits

# Получение тренировочной и тестовой выборки

In [57]:
X_train, y_train = train_df.iloc[:, :-1], train_df.iloc[:, -1]
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = test_df.to_numpy()

# `is_iterable`
Вспомогательная функция, которая проверяет то, что `obj` - это `iterable`.

In [58]:
def is_iterable(obj):
    try:
        iter(obj)
        return True
    except TypeError:
        return False

# `train_and_test`
Обобщающая функция, которая проводит обучение гиперпараметров (если это нужно), тренировку лучшей схемы, предсказание целевой характеристики и подсчет уверенности данного предсказания для тестовой выборки и записывает результат в csv-файл.

In [59]:
from sklearn.metrics import accuracy_score

def train_and_test(scheme, result_filename: str, hyperparams_metric=accuracy_score):
    if is_iterable(scheme):
        logging.info('HYPERPARAMS TUNING')
        scheme, hyperparams_tuning_metric = tune_hyperparams(schemas=scheme, X=X_train, y=y_train,
                                                             metric=accuracy_score)
        logging.info('HYPERPARAMS TUNING: ' + scheme.hyperparams + '. METRIC: ' + str(hyperparams_tuning_metric))

    logging.info('TRAINING')
    scheme.fit(X=X_train, y=y_train)

    logging.info('PREDICTING')
    y_predict = scheme.predict(X=X_test)
    y_predict_proba = scheme.predict_proba(X=X_test)
    y_predict_df = pd.DataFrame(data={'class': y_predict, 'certainty': y_predict_proba})
    y_predict_df.to_csv(os.path.join('result', result_filename + '.csv'), header=True, index=False, mode='w')

    return scheme, y_predict

# Схема 1. KNN
* Нормировка каждых признаков по отдельности.
* Используются только 50% лучших признаков по score mutual info.
* В качестве детектора выбросов используется `LocalOutlierFactor`.
* В качестве финального классификатора используется метод KNN с учетом расстояния между объектами.
* Количество соседей выбирается с помощью тренировки гиперпараметров.

In [62]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler

schemes_knn = [Scheme(pipeline=Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('feature_selector', SelectPercentile(score_func=mutual_info_classif, percentile=50)),
    ('classifier', KNeighborsClassifier(n_neighbors=n, weights='distance'))
]), outlier_detector=LocalOutlierFactor(n_neighbors=n), hyperparams_str=f'number of neighbours = {n}') for n in range(1, 31)]
train_and_test(scheme=schemes_knn, result_filename='KNN')

INFO:root:HYPERPARAMS TUNING
DEBUG:root:number of neighbours = 1
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (345, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (345, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (347, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (325, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (334, 432)
DEBUG:root:metric = 0.7464285714285716
DEBUG:root:number of neighbours = 2
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (355, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (348, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (342, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (337, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (352, 432)
DEBUG:root:metric = 0.7607142857142857
DEBUG:root:number of neighbours = 3
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (361, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (369, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (367, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (362, 432)
DEBUG:root:CLEA

(<__main__.Scheme at 0x7f144b6884c0>, array([1, 3, 2, ..., 3, 3, 1]))

# Схема 2. MLP
* Нормировка каждых признаков по отдельности.
* Используются только 50% лучших признаков по score mutual info.
* В качестве детектора выбросов используется `LocalOutlierFactor`.
* В качестве финального классификатора используется многослойный перцептрон с 3-мя скрытыми слоями, количество нейронов в которых выбирается с помощью тренировки гиперпараметров.

In [63]:
from sklearn.neural_network import MLPClassifier 

schemes_mlp = []
for i in range(15, 17):
    for j in range(10, 12):
        for k in range(5, 7):
            schemes_mlp.append(Scheme(pipeline=Pipeline(steps=[
                ('scaler', MinMaxScaler()),
                ('feature_selector', SelectPercentile(score_func=mutual_info_classif, percentile=50)),
                ('classifier', MLPClassifier(
                                    hidden_layer_sizes=[50 * i, 50 * j, 50 * k],
                                    max_iter=1000,
                                ))]), outlier_detector=LocalOutlierFactor(n_neighbors=15), hyperparams_str=f'layers = ({i}, {j}, {k})'))
train_and_test(scheme=schemes_mlp, result_filename='MLP')

INFO:root:HYPERPARAMS TUNING
DEBUG:root:layers = (15, 10, 5)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (342, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (379, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (345, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (397, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (394, 432)
DEBUG:root:metric = 0.7982142857142858
DEBUG:root:layers = (15, 10, 6)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (342, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (379, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (345, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (397, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (394, 432)
DEBUG:root:metric = 0.7946428571428571
DEBUG:root:layers = (15, 11, 5)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (342, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (379, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (345, 432)
DEBUG:root:CLEARING OUTLIERS: (448, 432) -> (397, 432)
DEBUG:root:CLEARING OUTLIER

(<__main__.Scheme at 0x7f144b748100>, array([1, 3, 2, ..., 3, 3, 1]))

# Схема 3. Дерево решений
* Нормировка каждых признаков по отдельности.
* Используются только 50% лучших признаков по score mutual info.
* В качестве детектора выбросов используется `Isolation Forest`.
* В качестве финального классификатора используется дерево решений.

In [64]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import IsolationForest

scheme_tree = Scheme(pipeline=Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('feature_selector', SelectPercentile(score_func=mutual_info_classif, percentile=50)),
    ('classifier', DecisionTreeClassifier(criterion='entropy', min_samples_split = 0.05, ))
]), outlier_detector=IsolationForest(n_jobs=-1))    
train_and_test(scheme=scheme_tree, result_filename='Decision tree')

INFO:root:TRAINING
INFO:root:CLEARING OUTLIERS: (560, 432) -> (506, 432)
INFO:root:PREDICTING


(<__main__.Scheme at 0x7f144b58bf40>, array([1, 3, 2, ..., 3, 3, 1]))

# Схема 4. Логистическая регрессия
* Нормировка каждых признаков по отдельности.
* Используются только 50% лучших признаков по score mutual info.
* В качестве финального классификатора используется логистическая регрессия.

In [66]:
from sklearn.linear_model import LogisticRegression

scheme_lr = Scheme(
    pipeline=Pipeline(
        steps=[
            ("scaler", MinMaxScaler()),
            ('feature_selector', SelectPercentile(score_func=mutual_info_classif, percentile=50)),  # normalize each feature independently
            ("classifier", LogisticRegression(max_iter=1000)),
        ]
    )
)
train_and_test(scheme=scheme_lr, result_filename='Logistic regression')

INFO:root:TRAINING
INFO:root:PREDICTING


(<__main__.Scheme at 0x7f144b589090>, array([1, 3, 2, ..., 3, 3, 1]))